In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools as itr
import warnings
warnings.filterwarnings("ignore")
import random as rd
import math as mt
import functions as fc
import time

# SIMULATED ANNEALING

In [ ]:
def Simulated_Annealing(*,x=2,y=1,T0=1000,M=300,N=15,alpha=0.85,k=0.1) :
    
    start_time = time.time() # start time (timing purposes)

    temp = []     # to plot the temprature
    obj_val = []  # to plot the obj val reached at the end of each m (small M)
    
    for i in range(M): # how many times to decrease the temp.
        for j in range(N): # for each m, how many neighborhood searche
            
            x_temporary,y_temporary=fc.improve(x,y,k)

            x,y,f=fc.update(x,y,x_temporary,y_temporary,T0)
            
        temp.append(T0)
        obj_val.append(f)

        T0 = alpha*T0
        
    print("Execution Time in Seconds:",time.time() - start_time) # exec. time
            
    return (x,y,obj_val,temp)

In [ ]:
x,y,obj_val,tempature=Simulated_Annealing()
print()
print("x is: %0.3f" % x)
print("y is: %0.3f" % y)
print("obj val is: %0.3f" % obj_val[-1])

In [ ]:
temp_for_plot=1000
plt.plot(tempature,obj_val)
plt.title("F at Temperature Values")
plt.xlabel("Temperature")
plt.ylabel("F")

plt.xlim(temp_for_plot,0)
plt.xticks(np.arange(min(tempature),max(tempature),100))
plt.yticks()

plt.show()

# TABU SEARCH

In [ ]:
### --> DYNAMIC TABU LIST <-- ###
### --> Short-term and long-term memories <-- ###


Dist = pd.DataFrame([[0,1,2,3,1,2,3,4],[1,0,1,2,2,1,2,3],[2,1,0,1,3,2,1,2],
                      [3,2,1,0,4,3,2,1],[1,2,3,4,0,1,2,3],[2,1,2,3,1,0,1,2],
                      [3,2,1,2,2,1,0,1],[4,3,2,1,3,2,1,0]],
                    columns=["A","B","C","D","E","F","G","H"],
                    index=["A","B","C","D","E","F","G","H"])

Flow = pd.DataFrame([[0,5,2,4,1,0,0,6],[5,0,3,0,2,2,2,0],[2,3,0,0,0,0,0,5],
                      [4,0,0,0,5,2,2,10],[1,2,0,5,0,10,0,0],[0,2,0,2,10,0,5,1],
                      [0,2,0,2,0,5,0,10],[6,0,5,10,0,1,10,0]],
                    columns=["A","B","C","D","E","F","G","H"],
                    index=["A","B","C","D","E","F","G","H"])


X0 = ["D","A","C","B","G","E","F","H"]

Initial_Solustion = X0[:] # For printing it at the end

In [ ]:
def Tabu(distance,flow,x,*,M=60,L=10):
    
    start_time = time.time() # start time (timing purposes)
    
    Tabu_List = np.empty((0,len(x)+1))
    One_Final_Guy_Final = []
    Save_Solutions_Here = np.empty((0,len(x)+1))
    Iterations = 1
    
    for i in range(M):

        All_N_for_i = fc.creat_neighbours(x)
        OF_Values_all_N=fc.neighbours_cost(x,distance,flow,All_N_for_i)

        # Ordered OF of neighborhood, sorted by OF value
        OF_Values_all_N_Ordered = np.array(sorted(OF_Values_all_N,key=lambda y: y[0]))

        # Check if solution is already in Tabu list, if yes, choose the next one
        Current_Sol,Tabu_List=fc.best_solution(OF_Values_all_N_Ordered,Tabu_List,L)
        
        Save_Solutions_Here = np.vstack((Current_Sol,Save_Solutions_Here)) # Save solutions, which is the best in each run

        # In order to "kick-start" the search when stuck in a local optimum, for diversification
        L,x,Iterations=fc.dynamic_list(x,Current_Sol,Iterations,L)

    One_Final_Guy_Final = fc.final_soluction(Save_Solutions_Here)[np.newaxis]
    
    print("Execution Time in Seconds:",time.time() - start_time) # exec. time
    
    return One_Final_Guy_Final[0]   

In [ ]:
result=Tabu(Dist,Flow,X0)
print()
print("DYNAMIC TABU LIST")
print()
print("Initial Solution:",Initial_Solustion)
print("Initial Cost:", fc.cost(Dist,Flow,Initial_Solustion))
print()
print("Min in all Iterations:",result[1:])
print("The Lowest Cost is:",result[0])

# GENETIC ALGORITHM

In [ ]:
def Genetic(x_y_string,*,M=80,N=120,pc=1,pm=0.3):
    
    start_time = time.time() # start time (timing purposes)

    # create an empty array to store a solution from each generation
    best_of_a_generation = np.empty((0,len(x_y_string)+1))

    # so now, pool_of_solutions, has n (population) chromosomes
    pool_of_solutions = fc.initialisez(x_y_string,N)

    for i in range(M): # do it n (generation) times

        new_population,new_population_with_obj_val=fc.create_population(x_y_string,N,pool_of_solutions,pc,pm)

        # we replace the initial (before) population with the new one (current generation)
        pool_of_solutions = new_population

        # for each generation we want to find the best solution in that generation
        sorted_best_for_plotting = np.array(sorted(new_population_with_obj_val,key=lambda x:x[0]))

        # since we sorted them from best to worst the best in that generation would be the first solution in the array
        best_of_a_generation = np.vstack((best_of_a_generation,sorted_best_for_plotting[0]))      
    
    # for this array of last population (convergence), there is a best solution so we sort them from best to worst
    sorted_last_population = np.array(sorted(new_population_with_obj_val,key=lambda x:x[0]))
    sorted_best_of_a_generation = np.array(sorted(best_of_a_generation,key=lambda x:x[0]))
    
    print("Execution Time in Seconds:",time.time() - start_time) # exec. time
    
    return (sorted_last_population,sorted_best_of_a_generation,best_of_a_generation,sorted_best_of_a_generation)


In [ ]:
# x and y decision variables' encoding
# 13 genes for x and 13 genes for y (arbitrary number)
x_y_string = np.array([0,1,0,0,0,1,0,0,1,0,1,1,1,0,1,1,1,0,0,1,0,1,1,0,1,1]) # initial solution
M=80

result=Genetic(x_y_string)

In [ ]:
# since we sorted them from best to worst
#the best would be the first solution in the array so index [0] of the "sorted_last_population" array
best_string_convergence = result[0][0]
best_string_overall = result[1][0]

print("Final Solution (Convergence):",best_string_convergence[1:]) # final solution entire chromosome
print("Encoded X (Convergence):",best_string_convergence[1:14]) # final solution x chromosome
print("Encoded Y (Convergence):",best_string_convergence[14:]) # final solution y chromosome
print()
print("Final Solution (Best):",best_string_overall[1:]) # final solution entire chromosome
print("Encoded X (Best):",best_string_overall[1:14]) # final solution x chromosome
print("Encoded Y (Best):",best_string_overall[14:]) # final solution y chromosome

In [ ]:
# to decode the x and y chromosomes to their real values
# the "genf.objective_value" function returns 3 things -->
# [0] is the x value
# [1] is the y value
# [2] is the obj val for the chromosome
# we use "best_string[1:]" because we don't want to include the first element [0]
# because it's the obj val, which is not a part of the actual chromosome
final_solution_convergence = fc.fitness(best_string_convergence[1:])
final_solution_overall = fc.fitness(best_string_overall[1:])

print("Decoded X (Convergence):",round(final_solution_convergence[0],5)) # real value of x
print("Decoded Y (Convergence):",round(final_solution_convergence[1],5)) # real value of y
print("f Value - Convergence:",round(final_solution_convergence[2],5)) # obj val of final chromosome
print()
print("Decoded X (Best):",round(final_solution_overall[0],5)) # real value of x
print("Decoded Y (Best):",round(final_solution_overall[1],5)) # real value of y
print("f Value - Best in Generations:",round(final_solution_overall[2],5)) # obj val of final chromosome

In [ ]:
### FOR PLOTTING THE BEST SOLUTION FROM EACH GENERATION ###

best_obj_val_convergence = best_string_convergence[0] # what the actual final answer is

best_obj_val_overall = best_string_overall[0]


plt.plot(result[2][:,0]) # check line 176
plt.axhline(y=best_obj_val_convergence,color='r',linestyle='--')

plt.axhline(y=best_obj_val_overall,color='r',linestyle='--')

plt.title("f Reached Through Generations",fontsize=20,fontweight='bold')
plt.xlabel("Generation",fontsize=18,fontweight='bold')
plt.ylabel("f",fontsize=18,fontweight='bold')
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')


if result[3][-1][0] > 2:
    k = 0.8
elif result[3][-1][0] > 1:
    k = 0.5
elif result[3][-1][0] > 0.5:
    k = 0.3
elif result[3][-1][0] > 0.3:
    k = 0.2
else:
    k = 0.1

xyz1 = (M/2.4,best_obj_val_convergence)
xyzz1 = (M/2.2,best_obj_val_convergence+k)

plt.annotate("At Convergence: %0.5f" % best_obj_val_convergence,xy=xyz1,xytext=xyzz1,
             arrowprops=dict(facecolor='black',shrink=1,width=1,headwidth=5),
             fontsize=12,fontweight='bold')


xyz2 = (M/6,best_obj_val_overall)
xyzz2 = (M/5.4,best_obj_val_overall+(k/2))

plt.annotate("Minimum Overall: %0.5f" % best_obj_val_overall,xy=xyz2,xytext=xyzz2,
             arrowprops=dict(facecolor='black',shrink=1,width=1,headwidth=5),
             fontsize=12,fontweight='bold')


plt.show()